Libraries

In [1]:
import cv2
import mediapipe as mp
import numpy as np

import pandas as pd  # For collecting data for research


# import time
# from scipy.spatial import distance as dis


# Only for data visualisation and exploration purposes, not to be included in practical use.
# import seaborn as sns
# import matplotlib.pyplot as plt

Objects

In [96]:
# Drowsiness detection using face landmark detection
mp_sol = mp.solutions
mp_drawing = mp_sol.drawing_utils
mp_drawing_styles = mp_sol.drawing_styles
mp_face_mesh = mp_sol.face_mesh
facemesh = mp_face_mesh.FaceMesh(max_num_faces = 1)
mp_holistic = mp_sol.holistic

landmark_style = mp_drawing.DrawingSpec((0,255,0), thickness=1, circle_radius=1)
connection_style = mp_drawing.DrawingSpec((0,0,255), thickness=1, circle_radius=1)

Data

In [29]:
FACE = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 
        152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
LIPS = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 
        88, 95, 185, 40, 39, 37, 0, 267, 269, 270, 409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78]
LOWER_LIPS = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS = [185, 40, 39, 37, 0, 267, 269, 270, 409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 

# Left eyes indices 
LEFT_EYE = [362, 398, 384, 385, 386, 387, 388, 466, 382, 381, 380, 374, 373, 390, 249, 263]
LEFT_EYEBROW = [336, 296, 334, 293, 300, 276, 283, 282, 295, 285]

# Right eyes indices
RIGHT_EYE = [33, 246, 161, 160, 159, 158, 157, 173, 7, 163, 144, 145, 153, 154, 155, 133]  
RIGHT_EYEBROW = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46]

EYES = [362, 398, 384, 385, 386, 387, 388, 466, 382, 381, 380, 374, 373, 390, 249, 263, 
        33, 246, 161, 160, 159, 158, 157, 173, 7, 163, 144, 145, 153, 154, 155, 133] 

LEFT_IRIS = [469, 470, 471, 472]
RIGHT_IRIS = [474, 475, 476, 477]

IRISES = [(474, 475), (475, 476), (476, 477), (477, 474), (469, 470), (470, 471), (471, 472), (472, 469)]

Functions

In [4]:
def eu_dist(pt1,pt2):
    x = np.square(pt2[0] - pt1[0])
    y = np.square(pt2[1] - pt1[1])
    z = np.square(pt2[2] - pt1[2])
    dist = np.sqrt(x + y + z)
    return(dist)


def blinked(le_nll, re_nll):
    left_v, right_v, ratio_le, ratio_re = 0, 0, 0, 0
    if len(le_nll) == 0 or len(re_nll) == 0:
        l_val, r_val = 0, 0
    
    else:
        # h for horizontal, v for vertical
        left_h = eu_dist(le_nll[0], le_nll[15])
        right_h = eu_dist(re_nll[0], re_nll[15])
        i = 1
        while i < 8:
            left_v += eu_dist(le_nll[i], le_nll[i + 7])
            right_v += eu_dist(re_nll[i], re_nll[i + 7])
            i += 1

        ratio_le = left_v/(7*left_h)
        ratio_re = right_v/(7*right_h)
        
        if ratio_le > 0.20:    # citation needed in value of EAR
            l_val = 1
        else:
            l_val = 0

        if ratio_re > 0.20:
            r_val = 1
        else:
            r_val = 0
    
#     return((ratio_le, ratio_re))
    return((l_val, r_val))


def aplist(le_nll, re_nll):
    le, re = [],[]
    if len(le_nll) == 0 or len(re_nll) == 0:
        le, re = [], []
    else:
        le.append(le_nll[2])
        le.append(le_nll[14])
        le.append(le_nll[8])
        le.append(le_nll[9])
        le.append(le_nll[10])
        le.append(le_nll[11])
        le.append(le_nll[12])
        le.append(le_nll[15])
        le.append(le_nll[7])
        le.append(le_nll[6])
        le.append(le_nll[5])
        le.append(le_nll[4])
        le.append(le_nll[3])
        le.append(le_nll[13])
        le.append(le_nll[0])
        le.append(le_nll[1])

        re.append(re_nll[1])
        re.append(re_nll[15])
        re.append(re_nll[12])
        re.append(re_nll[11])
        re.append(re_nll[10])
        re.append(re_nll[9])
        re.append(re_nll[8])
        re.append(re_nll[14])
        re.append(re_nll[0])
        re.append(re_nll[13])
        re.append(re_nll[3])
        re.append(re_nll[4])
        re.append(re_nll[5])
        re.append(re_nll[6])
        re.append(re_nll[7])
        re.append(re_nll[2])
    
    return(le, re)

Driver Facial Expression Monitoring and Driver Pose Detection

In [115]:
cap = cv2.VideoCapture("ec1.mp4")

frame_no, drowsy, la = 0, 0, []
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        frame_no += 1
        success, frame = cap.read()
        frame = cv2.flip(frame, 1)
        ih, iw, ic = frame.shape
        le_nll, re_nll, l_iris, r_iris = [], [], [], []
        
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Make detection
        results_fm = facemesh.process(frame)
        results_hol = holistic.process(frame)
        
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        if results_fm.multi_face_landmarks:
            for flms in results_fm.multi_face_landmarks:
                for id, lms in enumerate(flms.landmark):
                    if id in LEFT_EYE:
                        x, y, z = lms.x*iw, lms.y*ih, lms.z
                        le_nll.append((x, y, z))                        
                        
                    elif id in RIGHT_EYE:
                        x, y, z = lms.x*iw, lms.y*ih, lms.z
                        re_nll.append((x, y, z))

                    elif id in LEFT_IRIS:
                        x, y, z = lms.x*iw, lms.y*ih, lms.z
                        l_iris.append((x, y, z)) 
                    
                    elif id in RIGHT_IRIS:
                        x, y, z = lms.x*iw, lms.y*ih, lms.z
                        r_iris.append((x, y, z)) 
                        
                # Eyes and Face Oval
                mp_drawing.draw_landmarks(frame, results_hol.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                          landmark_drawing_spec = None, connection_drawing_spec = mp_drawing_styles
                                          .get_default_face_mesh_contours_style())
#                 # Left Hand Landmarks
#                 mp_drawing.draw_landmarks(frame, results_hol.left_hand_landmarks, mp_sol.hands_connections.HAND_CONNECTIONS,
#                                           mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1),
#                                           mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=1))
#                 # Right Hand Landmarks
#                 mp_drawing.draw_landmarks(frame, results_hol.right_hand_landmarks, mp_sol.hands_connections.HAND_CONNECTIONS,
#                                           mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1),
#                                           mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=1))
#                 # Pose Landmarks
#                 mp_drawing.draw_landmarks(frame, results_hol.pose_landmarks, mp_sol.pose.POSE_CONNECTIONS,
#                                           mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1),
#                                           mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=1))
                # Irises
                mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) 
                                      for p in results.multi_face_landmarks[0].landmark])

                (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRIS])
                (r_cx,r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            
                center_left = np.array([l_cx, l_cy], dtype=np.int32)
                center_right = np.array([r_cx, r_cy], dtype=np.int32)
            
                cv2.circle(frame, center_left, int(l_radius), (255, 0, 255), 1, cv2.LINE_AA)
                cv2.circle(frame, center_right, int(r_radius), (255, 0, 255), 1, cv2.LINE_AA)   


            status, instruct, color = "", "", (0,0,0)               
            a = aplist(le_nll, re_nll)
            a = blinked(a[0], a[1])

            if a[0] == 1 or a[1] == 1:
                status = "ALERT."
                color = (48, 255, 48)
                drowsy = 0

            else:
                drowsy += 1
                status = "BLINKED"
                color = (48, 255, 200)
                if drowsy > 10:
                    status = "DROWSY!!!"
                    color = (0,0,255)

     
# with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
#         frame = cv.flip(frame, 1)
#         rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)   #Mediapipe precisa do formato de cores RGB mas o OpenCV usa o BGR
#         img_h, img_w = frame.shape[:2]
#         results = face_mesh.process(rgb_frame)
#         if results.multi_face_landmarks:
#             mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])

#             (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
#             (r_cx,r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
                
                                    
        cv2.putText(frame, str(frame_no), (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (224, 224, 224), 3)
        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        cv2.imshow('MediaPipe', frame)
        if cv2.waitKey(5) & 0xFF == 27:
            break
                
cap.release()
cv2.destroyAllWindows()

In [120]:
cap = cv2.VideoCapture("test1.mp4")
with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   #Mediapipe precisa do formato de cores RGB mas o OpenCV usa o BGR
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])

            #transformar formas quadradas em círculos, função do OpenCV fornece círculos delimitadores com base nos pontos fornecidos.
            #minEnclosingCircle que retorna, o centro (x,y) e o raio dos círculos, os valores de retorno são de ponto flutuante, necessario transformá-los em int.
            (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx,r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)
            
            cv2.circle(frame, center_left, int(l_radius), (255, 0, 255), 1, cv2.LINE_AA)
            cv2.circle(frame, center_right, int(r_radius), (255, 0, 255), 1, cv2.LINE_AA)

                
                                    
        cv2.putText(frame, str(frame_no), (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (224, 224, 224), 3)
        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        cv2.imshow('MediaPipe', frame)
        if cv2.waitKey(5) & 0xFF == 27:
            break
                
cap.release()
cv2.destroyAllWindows()

In [99]:
# Release and destroy all windows
cap.release()
cv2.destroyAllWindows()

## 

### Experimental Area

In [92]:
la

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [71]:
results_hol.left_hand_landmarks??

In [75]:
cv2.minEnclosingCircle??

In [60]:
FACEMESH_CONTOURS
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_TESSELATIO N

SyntaxError: invalid syntax (1058137438.py, line 2)

In [22]:
df = pd.DataFrame(data, columns = ["Left_Eyes", "Right_Eyes"])
df

,Left_Eyes,Right_Eyes
0,0.158608,0.162505
1,0.133572,0.134264
2,0.134285,0.134780
3,0.128135,0.128929
4,0.131674,0.132947
...,...,...
2045,0.100987,0.106167
2046,0.099441,0.104686
2047,0.100485,0.105953
2048,0.104475,0.109868


In [23]:
df.to_csv("C:\\Users\\thepr\\Downloads\\rizecwithoutg.csv")

In [53]:
test3 and test1 , eo1 faulty